In [ ]:
import cv2 as cv
import numpy as np

def horizontal_flip(img, mask):
    return cv.flip(img, 0), cv.flip(mask, 0)

def vertical_flip(img, mask):
    return cv.flip(img, 1), cv.flip(mask, 1)

def horizontal_and_vertical_flip(img, mask):
    return cv.flip(img, -1), cv.flip(mask, -1)


def zoom_in(img, mask):
    row, col = img.shape
    img = img[56:500, 56:456]
    mask = mask[56:500, 56:456]
    return cv.resize(img, (col, row), fx=1, fy=1), cv.resize(mask, (col,row), fx=1, fy=1)

def zoom_out(img, mask):
    # Add zero padding to zoom in
    zero_img = np.zeros_like(img)
    zero_mask = np.zeros_like(mask)
    img = cv.resize(img, None, fx=0.8, fy=0.8)
    mask = cv.resize(mask, None, fx=0.8, fy=0.8)
    row, col = zero_img.shape

    center = (row//2, col//2)
    zero_img[center[0]-img.shape[0]//2:center[0]+img.shape[0]//2, center[1]-img.shape[1]//2:center[1]+img.shape[1]//2] = img
    zero_mask[center[0]-img.shape[0]//2:center[0]+img.shape[0]//2, center[1]-img.shape[1]//2:center[1]+img.shape[1]//2] = mask

    return zero_img, zero_mask


def translation(img, mask):
    rows, cols = img.shape
    tx = np.random.randint(-30, 30)
    ty = np.random.randint(-30, 30)
    M = np.float32([[1,0,tx],[0,1,ty]])
    return cv.warpAffine(img, M, (cols, rows)), cv.warpAffine(mask, M, (cols, rows))



In [ ]:
import numpy as np
import os
import cv2 as cv

root_path = "./train"

images_paths = sorted([root_path+"/images/"+i for i in os.listdir(root_path+"/images/")])
masks_paths = sorted([root_path+"/masks/"+i for i in os.listdir(root_path+"/masks/")])

print(images_paths)
print(masks_paths)

# Load all images and masks before the loop
imgs = []
masks = []
for i in range(len(images_paths)):
    imgs.append((cv.imread(images_paths[i], cv.IMREAD_GRAYSCALE)))
    masks.append((cv.imread(masks_paths[i], cv.IMREAD_GRAYSCALE)))

seed_value = 42
rng = np.random.default_rng(seed_value)

if not os.path.exists("./aug_images/"):
  os.mkdir("./aug_images/")

if not os.path.exists("./aug_masks/"):
  os.mkdir("./aug_masks/")


for i in range(len(images_paths)):
    img_select = np.random.randint(low=0, high=len(masks_paths))
    print("Image selected>", img_select)
    transform_select = np.random.randint(low=0, high=6)
    if(transform_select == 0):
      img, mask = horizontal_flip(imgs[img_select], masks[img_select])
    elif(transform_select == 1):
      img, mask = vertical_flip(imgs[img_select], masks[img_select])
    elif(transform_select == 2):
      img, mask = horizontal_and_vertical_flip(imgs[img_select], masks[img_select])
    elif(transform_select == 3):
      img, mask = zoom_in(imgs[img_select], masks[img_select])
    elif(transform_select == 4):
      img, mask =zoom_out(imgs[img_select], masks[img_select])
    else:
      img, mask = translation(imgs[img_select], masks[img_select])




    cv.imwrite(f"./aug_images/augmented_{i}.jpg", img)
    cv.imwrite(f"./aug_masks/augmented_{i}.png", mask)